In [ ]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Dropout,Flatten,BatchNormalization,Activation
from tensorflow.keras.applications import NASNetLarge,VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
folder = './drive/My Drive/Colab/datasets'
batch_size1=20

In [ ]:
train_label_df = pd.read_csv(folder + '/All_Ratings.csv', delimiter=',', header=None, names=['id', 'score'])

In [ ]:
train_label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7816 entries, 0 to 7815
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      7816 non-null   object 
 1   score   7816 non-null   float64
dtypes: float64(1), object(1)
memory usage: 122.2+ KB


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./331, horizontal_flip = True,
                                   fill_mode = "nearest", zoom_range = 0.2,
                                   width_shift_range = 0.2, height_shift_range=0.2,
                                   rotation_range=10, validation_split= 0.3,
                                   preprocessing_function=preprocess_input) 

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_label_df, directory=folder + '/img', 
                                              x_col="id", y_col="score", has_ext=True, 
                                              class_mode='raw', target_size=(331, 331), 
                                              batch_size=batch_size1,subset='training',
                                              shuffle=True, interpolation='lanczos')

Found 5472 validated image filenames.


In [ ]:
validation_generator = train_datagen.flow_from_dataframe(dataframe=train_label_df, directory=folder + '/img', 
                                              x_col="id", y_col="score", has_ext=True, 
                                              class_mode='raw', target_size=(331, 331), 
                                              batch_size=batch_size1,subset='validation',
                                              shuffle=True, interpolation='lanczos')

Found 2344 validated image filenames.


In [ ]:
pretrained_model = NASNetLarge(
      weights=None, 
      include_top=False, 
      input_shape=(331, 331, 3))

model = tf.keras.Sequential([
            pretrained_model,
            GlobalMaxPooling2D(),
            Dense(1000),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(250),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(1, activation='linear')])

print("Done.")

Done.


In [ ]:
checkpoint = ModelCheckpoint('./drive/My Drive/Colab/GPU/model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='min')

In [ ]:
model.load_weights('./drive/My Drive/Colab/1Cycle/model-138-0.630001-0.430507.h5')

In [ ]:
for layer in model.layers[:-10]:
    layer.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_max_pooling2d (Global (None, 4032)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              4033000   
_________________________________________________________________
batch_normalization (BatchNo (None, 1000)              4000      
_________________________________________________________________
activation_260 (Activation)  (None, 1000)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2

In [ ]:
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['mean_absolute_error','mean_absolute_percentage_error'])

In [ ]:
model.fit(train_generator,
        steps_per_epoch = len(train_generator),
        epochs=100,
        validation_data=validation_generator,
        validation_steps=len((validation_generator)),
        callbacks=[checkpoint])

Epoch 1/100
274/274 [==============================] - ETA: 0s - loss: 2.3981 - mean_absolute_error: 1.2361 - mean_absolute_percentage_error: 24.5997
Epoch 00001: val_loss improved from inf to 134760142261982697267952226846900224.00000, saving model to ./drive/My Drive/Colab/GPU/model-001-2.398095-134760142261982697267952226846900224.000000.h5
274/274 [==============================] - 2661s 10s/step - loss: 2.3981 - mean_absolute_error: 1.2361 - mean_absolute_percentage_error: 24.5997 - val_loss: 134760142261982697267952226846900224.0000 - val_mean_absolute_error: 367093912805834752.0000 - val_mean_absolute_percentage_error: 7425193887150374912.0000
Epoch 2/100
274/274 [==============================] - ETA: 0s - loss: 1.9440 - mean_absolute_error: 1.1153 - mean_absolute_percentage_error: 22.4673
Epoch 00002: val_loss improved from 134760142261982697267952226846900224.00000 to 40431166248901934051895293524312064.00000, saving model to ./drive/My Drive/Colab/GPU/model-002-1.943996-4043